In [1]:
from siphon.catalog import TDSCatalog
from pathlib import Path
from google.cloud import storage
import io, os
import urllib.request
from urllib.parse import urlparse
from pathlib import Path

The project id and service account key are mine. You would need to get your own, though you could perhaps use the same project key
if you can get a service account under that project.

In [2]:
project_id = 'xxx-xxx-xxx-xxx-xxx-xxx'
bucket_name = 'noaa-oar-cefi-regional-mom6'
storage_client = storage.Client.from_service_account_json('your-key-file.json')
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
This read from the thredds catalog and uses the HTTPServer access URL to download each file. You should rework this to scan your local
directories with the file and transfer the files directly.

In [3]:
match = 'cefi_portal'
catalog_root = 'https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/catalog.xml'

catalog_prefix_index = catalog_root.find(match)
catalog_replace = catalog_root[:catalog_prefix_index + len(match) + 1]
print(catalog_replace)

https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/


In [16]:
def get_datasets(root, total):
    if root.datasets and 'latest' not in root.catalog_url and 'r20240213' not in root.catalog_url:
        downloads = 0
        for dataset in root.datasets:
            access_url = root.datasets[dataset].access_urls['HTTPServer']
            dataset_prefix = access_url.find(match)
            url_replace = access_url[:dataset_prefix + len(match) + 1]
            outfile = access_url.replace(url_replace,'')
            downloads = downloads  + 1
            file_path = Path(outfile)
            file_path.parent.mkdir(parents=True, exist_ok=True)
            destination_blob_name = outfile
            blob = bucket.blob(destination_blob_name)
            if not blob.exists():
                urllib.request.urlretrieve(access_url, outfile)
                blob.upload_from_filename(outfile, content_type='application/x-netcdf')
                os.remove(outfile)
        if downloads > 0:
            total = total + downloads
            print(root.catalog_url)
            print(f'Downloaded {downloads} files.')
    for catalog in root.catalog_refs:
        child = root.catalog_refs[catalog].follow()
        total = get_datasets(child, total)
    return total

In [17]:
cat = TDSCatalog(catalog_root)
total = 0
total = get_datasets(cat, total)
print(f'Total number of files transferred = {total}')

https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/hindcast/daily/raw/r20230520/catalog.xml
Downloaded 24 files.
https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/hindcast/daily/regrid/r20230520/catalog.xml
Downloaded 20 files.
https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/hindcast/monthly/raw/r20230520/catalog.xml
Downloaded 492 files.
https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/hindcast/monthly/regrid/r20230520/catalog.xml
Downloaded 452 files.
https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/seasonal_forecast/monthly/raw/r20250212/catalog.xml
Downloaded 4 files.
https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northwest_atlantic/full_domain/seasonal_forecast/mo